In [1]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 
from typing import Any, Optional, List

import qubx
%qubxd dev 

%load_ext autoreload
%autoreload 2

from qubx import lookup, logger
from qubx.core.strategy import IPositionGathering, IStrategy, PositionsTracker, StrategyContext, TriggerEvent
from qubx.pandaz.utils import *
import qubx.pandaz.ta as pta

# - - - - - - - - - - - - - - - - - - - - - - - -
from qubx.data.readers import CsvStorageDataReader, MultiQdbConnector, AsTimestampedRecords, AsQuotes, RestoreTicksFromOHLC, AsPandasFrame, RestoreTicksFromOHLC, AsOhlcvSeries
from qubx.core.basics import Deal, Instrument, Order, Position, Signal
from qubx.utils.charting.lookinglass import LookingGlass
from qubx.core.utils import recognize_time, time_to_str

from qubx.backtester.simulator import simulate
from qubx.core.metrics import tearsheet, chart_signals
from qubx.trackers.riskctrl import StopTakePositionTracker
from qubx.trackers.sizers import FixedSizer, FixedRiskSizer, FixedLeverageSizer

 >  [dev] installing cython rebuilding hook

⠀⠀⡰⡖⠒⠒⢒⢦⠀⠀   
⠀⢠⠃⠈⢆⣀⣎⣀⣱⡀  QUBX | Quantitative Backtesting Environment 
⠀⢳⠒⠒⡞⠚⡄⠀⡰⠁         (c) 2024, ver. Dev
⠀⠀⠱⣜⣀⣀⣈⣦⠃⠀⠀⠀ 
        


In [3]:
signals = pd.read_csv("../tests/data/generated_signals.csv", index_col="timestamp", parse_dates=True)
print(signals.index[0], signals.index[-1])

exchange = "BINANCE.UM"
symbol = "BTCUSDT"
# reader = CsvStorageDataReader("../tests/data/csv")
# ohlc = reader.read("BTCUSDT_ohlcv_M1", start='2023-08-13', stop='2024-04-30', transform=AsOhlcvSeries())
qdb = MultiQdbConnector('xlydian-data')
ohlc = qdb.read(f"{ exchange }:{symbol}", start=signals.index[0], stop='2024-05-01', transform=AsPandasFrame())

In [4]:
S = pd.DataFrame({'BTCUSDT': {
    pd.Timestamp('2023-08-27 14:14:45.431000'): +1,
    pd.Timestamp('2023-08-27 14:37:42.621000'): +1,
    pd.Timestamp('2023-08-27 16:16:00.451000'): +1,
    pd.Timestamp('2023-08-27 16:49:18.924000'): +1,
    pd.Timestamp('2023-08-27 19:07:23.468000'): +1,
    pd.Timestamp('2023-08-28 12:09:00.892000'): +1,
    pd.Timestamp('2023-08-28 12:14:24.860000'): +1,
    pd.Timestamp('2023-08-28 12:26:43.505000'): +1,
    pd.Timestamp('2023-08-28 23:19:48.408000'): +1,
    pd.Timestamp('2023-08-29 14:02:56.179000'): +1,
    pd.Timestamp('2023-08-29 14:03:58.530000'): +1,
    pd.Timestamp('2023-08-29 14:21:00.859000'): +1,
    pd.Timestamp('2023-08-29 14:25:55.093000'): +1,
    pd.Timestamp('2023-08-29 14:26:48.430000'): +1}})

In [7]:
# S = signals['2023-08-27 14:00':'2023-08-29 14:30']
# S = signals
rep = simulate(
    { "liq_buy_bounces": [ S, StopTakePositionTracker(2.5, 0.5, FixedLeverageSizer(0.1), "client")]},
    {f"{exchange}:{symbol}" : ohlc}, 
    10000, [exchange + ":" + symbol], dict(type="ohlc", timeframe="1Min"), "1Min -1Sec", "vip9_usdt", 
    S.index[0], S.index[-1] + pd.Timedelta('5Min'), debug='DEBUG')

2024-09-06 14:28:27.985 [ 🐞 ] 2023-08-27 14:14:45.431000 Initiating simulated trading for binance.um for 10000 x 1.0 in USDT...
2023-08-27 14:14:45.431 [ℹ️] SimulatedData.binance.um initialized
2023-08-27 14:14:45.431 [🐞] Using pre-generated signals:
 BTCUSDT    14

2023-08-27 14:14:45.431 [🐞] Triggering strategy on every 1Min bar after 
2023-08-27 14:14:45.431 [🐞] (StrategyContext) Subscribing to ohlc updates using {'timeframe': '1Min', 'nback': 1} for 
	['BTCUSDT'] 
2023-08-27 14:14:45.431 [🐞] SimulatedExchangeService :: subscribe :: BTCUSDT :: ohlc


Simulating:   0%|          | 0/173522.999 [00:00<?, ?s/s]

2023-08-27 14:14:45.431 [ℹ️] SimulatedExchangeService :: run :: Simulation started at 2023-08-27 14:14:45.431000
2023-08-27 14:14:45.431 [🐞] Initializing chunks for each loader
2023-08-27 14:14:45.431 [🐞] Initializing chunks for each loader
2023-08-27 14:15:00.001 [🐞] Invoking SignalsProxy on_fit('2023-08-27 14:14:45.431000', 'None')
2023-08-27 14:15:00.001 [🐞] SignalsProxy is fitted
2023-08-27 14:15:00.001 [🐞] ClientSideRiskController new signal received Target for SignalsProxy 26120.10 +1.000000 BTCUSDT stop: 25989.549249999996 take: 26773.153749999994 on BINANCE.UM [] -> 0.038 at 2023-08-27T14:15:00.001000000 for BTCUSDT take: 26773.153749999994 stop: 25989.549249999996
2023-08-27 14:15:00.001 [🐞] (StrategyContext) sending market buy for 0.038 of BTCUSDT @ None ...
2023-08-27 14:15:00.001 [🐞] [OMS] BTCUSDT - SIM-ORDER-BTCUSDT-100001 MARKET BUY 0.038 executed at 26120.149999999998
2023-08-27 14:15:00.001 [🐞]   ::  traded 0.038 for BTCUSDT @ 26120.149999999998 -> 0.00
2023-08-27 14:15

2023-08-29 14:26:59.999 [🐞] ClientSideRiskController starts tracking Target for SignalsProxy 27095.90 +1.000000 BTCUSDT stop: 26960.470250000002 take: 27773.348749999997 on BINANCE.UM [] -> 0.036 at 2023-08-29T14:26:59.999000000 of BTCUSDT take: 27773.348749999997 stop: 26960.470250000002
2023-08-29 14:26:59.999 [🐞] Executed -0.001 @ 27095.850000000002 of BTCUSDT for order SIM-ORDER-BTCUSDT-100009
2023-08-29 14:26:59.999 [🐞] BTCUSDT: Adjusting position from 0.037 to 0.036 : [SIM-ORDER-BTCUSDT-100009] MARKET SELL 0.001 of BTCUSDT  (gtc) [CLOSED]
2023-08-29 14:26:59.999 [ℹ️] SimulatedExchangeService :: run :: Simulation finished at 2023-08-29 14:26:48.430000


In [6]:
rep[0].signals_log

,instrument_id,exchange_id,signal,target_position,reference_price,price,take,stop,group,comment,service
timestamp,,,,,,,,,,,
2023-08-27 14:15:00.001,BTCUSDT,BINANCE.UM,1,0.038,26120.1,None,26773.15375,25989.54925,SignalsProxy,,False
2023-08-27 14:37:59.999,BTCUSDT,BINANCE.UM,1,0.038,26163.2,None,26817.33125,26032.43375,SignalsProxy,,False
2023-08-27 16:16:24.000,BTCUSDT,BINANCE.UM,1,0.038,26138.5,None,26792.01375,26007.85725,SignalsProxy,,False
2023-08-27 16:49:24.000,BTCUSDT,BINANCE.UM,1,0.038,26148.4,None,26802.16125,26017.70775,SignalsProxy,,False
2023-08-27 19:07:24.000,BTCUSDT,BINANCE.UM,1,0.038,26157.0,None,26810.97625,26026.26475,SignalsProxy,,False
2023-08-28 00:52:36.000,BTCUSDT,BINANCE.UM,0,0.000,25971.0,None,NaN,NaN,Risk Manager,Stop triggered,False
2023-08-28 12:09:24.000,BTCUSDT,BINANCE.UM,1,0.038,26070.3,None,26722.10875,25939.99825,SignalsProxy,,False
2023-08-28 12:14:36.000,BTCUSDT,BINANCE.UM,1,0.038,26074.5,None,26726.41375,25944.17725,SignalsProxy,,False
2023-08-28 12:26:59.999,BTCUSDT,BINANCE.UM,1,0.038,26209.1,None,26864.37875,26078.10425,SignalsProxy,,False


In [14]:
rep[0].executions_log

,instrument_id,side,filled_qty,price,commissions,commissions_quoted
timestamp,,,,,,
2023-08-27 14:15:00.001,BTCUSDT,buy,0.038,26120.15,0.168736,USDT
2023-08-27 14:37:59.999,BTCUSDT,sell,-0.038,26163.15,0.169014,USDT
2023-08-27 16:16:24.000,BTCUSDT,buy,0.038,26138.55,0.168855,USDT
2023-08-28 00:52:36.000,BTCUSDT,sell,-0.038,25970.95,0.167772,USDT
2023-08-28 12:09:24.000,BTCUSDT,buy,0.038,26070.35,0.168414,USDT
2023-08-28 19:23:36.000,BTCUSDT,sell,-0.038,25910.05,0.167379,USDT
2023-08-28 23:19:59.999,BTCUSDT,buy,0.038,26135.65,0.168836,USDT
2023-08-29 06:51:36.000,BTCUSDT,sell,-0.038,26004.45,0.167989,USDT
2023-08-29 14:02:59.999,BTCUSDT,buy,0.038,26120.05,0.168736,USDT


In [ ]:
tearsheet(rep)

In [15]:
chart_signals(rep[0], symbol, ohlc, '1Min',  show_signals=True)